In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'html.parser')
my_table = soup.find('table', class_ = 'sortable')

ths = my_table.find_all('th')
headings = [th.text.strip() for th in ths]

rows = my_table.find_all('tr')

postals = [[td.getText().rstrip() for td in rows[i].findAll('td')]
for i in range(len(rows))]

postals = pd.DataFrame(postals, columns = None)
postals.columns = headings

postals = postals[1:]

postals = postals[postals.Borough != 'Not assigned']

postals = postals.reset_index(drop = True)
print(postals)

for i in range(1, len(postals)):
    if postals.Neighbourhood[i] == 'Not assigned':
        print(postals.Borough[i])
        postals.Neighbourhood[i] = postals.Borough[i]

    Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M6A        North York   
4           M7A  Downtown Toronto   
..          ...               ...   
98          M8X         Etobicoke   
99          M4Y  Downtown Toronto   
100         M7Y      East Toronto   
101         M8Y         Etobicoke   
102         M8Z         Etobicoke   

                                         Neighbourhood  
0                                            Parkwoods  
1                                     Victoria Village  
2                            Regent Park, Harbourfront  
3                     Lawrence Manor, Lawrence Heights  
4          Queen's Park, Ontario Provincial Government  
..                                                 ...  
98       The Kingsway, Montgomery Road, Old Mill North  
99                                Church and Wellesley  
100  Business reply mail Processing Centre

In [4]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_683e7e5dfdaa4d18855f9767f58c452e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='f0p7y4naDYPCW8GBeQxqLFjCwX6yjqG0zPlhBE52BphG',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_683e7e5dfdaa4d18855f9767f58c452e.get_object(Bucket='capstone-donotdelete-pr-qrjajsciva6iyp',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
joined = pd.merge(postals, df_data_1)

joined.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
